# Estrategias Evolutivas
## Computacion Evolutiva
### Eduardo Manuel Ceja Cruz

In [ ]:
# no correr si se tienen las librerias
!conda install -c fastai fastprogress

In [1]:
import math
import numpy as np

from typing import Tuple, List, Union, Dict
from random import choices,choice
from tabulate import tabulate
from fastprogress.fastprogress import progress_bar

La funcion de Ackley esta definida de la siguente manera:

$$
\min f(\vec{x}) = -20exp \left(-0.2 \sqrt{\frac{1}{n} \sum_{i=1}^n x^2_i} \right) - exp \left(\frac{1}{n}\sum_{i=1}^n cos(2\pi x_i) \right) + 20 + e

$$

In [2]:
MAX_DOMAIN = 30
MIN_DOMAIN = -MAX_DOMAIN

In [3]:
def read_input_from_file(filename : str = 'input.txt') -> Tuple[int, int, float, float]:
    try:
        with open(filename) as f:
            lines = f.readlines()
    
    except FileNotFoundError:
        print(f"El archivo {filename} no fue encontrado")
        return
    
    except Exception as e:
        print(f"Ha occurrido un error {e}")
        return
    
    n = int(lines.pop(0))
    mu, g = map(int, lines.pop(0).split())
    alpha, epsilon = map(float , lines.pop(0).split())
    return n, mu, g, alpha, epsilon

In [4]:
def std(M_list : list, mean : float) -> float:
    return np.sqrt(sum([(sol_i[-1] - mean)**2 for sol_i in M_list])/len(M_list))

In [5]:
def mean(sol_list : list) -> float:
    return sum([elem[-1] for elem in sol_list])/len(sol_list)

In [6]:
def f(x : List[float], n : int) -> float:
    coso1 = -0.2*np.sqrt(sum([xi**2 for xi in x[:n]])/n)
    coso2 = (sum([math.cos(2*math.pi*xi) for xi in x[:n]])/n)
    return -20*math.e**(coso1) -math.e**(coso2) + 20 + math.e

## Resolucion con $(1+1)$

In [7]:
def generate_initial_solution(n : int):
    return np.random.uniform(MIN_DOMAIN, MAX_DOMAIN, n)

In [8]:
def EE_1_plus_1(n : int, lim_x : List[List[float]], sigma0 : float = 0.5, G : int = 100,  r : int = 20, eps0 : float =0.01, c : float = 0.817):
    
    sigma = sigma0
    xt = generate_initial_solution(n)
    
    ft = -f(xt, n) # lo ponemos en menos porque estamos minimizando, en lugar de maximixar

    ms = 0
    # pb = progress_bar(range(G))
    for epoch in range(G):
        counter = 0
        if counter%r == 0:
            if ms/r > 1/5:
                sigma /= c
            elif ms / r < 1/5:
                sigma *= c

            if sigma < eps0:
                sigma = eps0
            ms = 0
        counter += 1
    
    z = np.random.normal(0, sigma, n)
    xt_mas_1 = np.add(xt, z)
    for i in range(n):
        if xt_mas_1[i] < lim_x[i][0]:
            xt_mas_1[i] = lim_x[i][0]
        elif xt_mas_1[i] > lim_x[i][1]:
            xt_mas_1[i] = lim_x[i][1]
    
    ft_mas_1 = -f(xt_mas_1, n) # lo ponemos en menos porque estamos minimizando, en lugar de maximixar

    if ft_mas_1 >= ft:
        xt = xt_mas_1
        ft = ft_mas_1
        ms += 1

    return xt, ft

## EStrategia ($\mu , \lambda$)

In [9]:
def initial_population(mu : int, n : int) -> List[List[float]]:
    padres = []
    for i in range(mu):
        p = np.concatenate((np.random.uniform(MIN_DOMAIN, MAX_DOMAIN, n) ,np.random.uniform(0,1, 1)))
        p = [p, -f(p, n)] # le ponemos un menos porque estamos minimizando
        padres.append(p)
    return padres

In [10]:
def cruza(padre_1 : List[float], padre_2 : List[float]):
    N = len(padre_1)

    z = np.random.uniform(0,1, N)
    child = np.array([0.] * N)

    for i in range(N):
        if z[i] < 0.5:
            child[i] = padre_1[i]
        else:
            child[i] = padre_2[i]

    return child

In [11]:
def mutacion(i : List[float], n : int, eps0 : float, tao : float):
    new_sigma = i[-1]*(np.exp(tao*np.random.normal(0,1,1)))

    if new_sigma[0] < eps0 :
        new_sigma[0] = eps0
    
    mutation_x = i[:n] + (new_sigma[0]*np.random.normal(0,1,n))

    mutation_x[mutation_x < MIN_DOMAIN] = MIN_DOMAIN
    mutation_x[mutation_x > MAX_DOMAIN] = MAX_DOMAIN

    return np.concatenate((mutation_x, new_sigma))

In [12]:
def create_individual(padre_1 : List[float], padre_2 : List[float], n : int, eps0 : float, tao : float) -> List[float]:
    x = cruza(padre_1, padre_2)
    x = mutacion(x, n, eps0, tao)
    
    return[x, -f(x,n)] # le ponemos el menos porque estamos minimizando

In [13]:
i1 = [-10.3, 7.84, 0.84]
i2 = [2.4, -3.84, 0.98]
create_individual(i1,i2, 2, 1/np.sqrt(2), 0.01)

[array([-8.14698103, -5.08385834,  0.84263623]), -15.492426950773847]

In [14]:
def mu_lambda(n : int, G : int = 100, eps0 : float = 0.01, mu : int = 100, lamb : int = 700, tao : float = None):

    if tao is None:
        tao = 1/np.sqrt(n)
    
    poblacion = initial_population(mu, n)
    index = range(mu)
    
    # pb = progress_bar(range(G))
    for epoch in range(G):
        hijos = []
        for i in range(lamb):
            padres = choices(index, k=2)
            padre_1 = poblacion[padres[0]][0]
            padre_2 = poblacion[padres[1]][0]
            hijo = create_individual(padre_1, padre_2, n, eps0, tao)
            hijos.append(hijo)
        
        hijos = sorted(hijos, key = lambda x : x[-1], reverse=True)
        poblacion = hijos[:mu].copy()
    
    return poblacion[0][0], -poblacion[0][1]

In [15]:
mu_lambda(2, 100)

(array([-0.00091883,  0.00059321,  0.01      ]), 0.0031252545631947903)

## Estrategia $(\mu + \lambda)$

In [16]:
def mu_mas_lambda(n : int, G : int = 100, eps0 : float = 0.01, mu : int = 100, lamb: int = 700, tao : float = None):

    if tao is None:
        tao = 1/np.sqrt(n)

    poblacion = initial_population(mu, n)
    index = range(mu)
    # pb = progress_bar(range(G))
    for epoch in range(G):
        hijos = []
        for i in range(lamb):
            padres = choices(index, k=2)
            padre_1 = poblacion[padres[0]][0]
            padre_2 = poblacion[padres[1]][0]
            hijo = create_individual(padre_1, padre_2, n, eps0, tao)
            hijos.append(hijo)
        
        poblacion.extend(hijos)
        poblacion = sorted(poblacion, key = lambda x : x[-1], reverse = True)
        poblacion = poblacion[:mu]
        # pb.comment(f'best sol : {poblacion[0][-1]}')
    return poblacion[0][0], -poblacion[0][1]

In [17]:
mu_mas_lambda(2,100)

(array([ 2.54543669e-05, -1.99570810e-05,  1.00000000e-02]),
 9.151380877314708e-05)

## Parte Estadistica

In [18]:
def parse_1_m_1_EE_params(params : dict) -> Tuple[Dict[str, Union[float, int]], Dict[str, Union[float, int]]]:
    ee1_m_1_params = {}
    
    ee1_m_1_params['n'] = params['n']
    
    if 'eps0' in params:
        ee1_m_1_params['eps0'] = params['eps0']

    if 'sigma0' in params:
        ee1_m_1_params['sigma0'] = params['sigma0']
    if 'G' in params:
        ee1_m_1_params['G'] = params['G']
    if 'r' in params:
        ee1_m_1_params['r'] = params.pop('r')
    if 'c' in params:
        ee1_m_1_params['c'] = params.pop('c')
    
    return ee1_m_1_params, params

In [19]:
def format_data(data : list, columns  : List[str] = ['Estrategia Evolutiva','Mejor Solucion', 'Peor Solucion', 'Media', 'Mediana','Desviacion Estandar']) -> str:
    return tabulate(data, headers=columns)

In [24]:
def statdistical(M : int, ee1_m_1_lim_x : List[List[float]], **kwargs) -> Union[str, Tuple[str, list, list, list]]:
    params = kwargs
    return_sols = False
    if 'n' not in params:
        params['n'] = 2
    
    if 'return_sols' in params:
        return_sols = params.pop('return_sols')
    ee1_m_1_params, params = parse_1_m_1_EE_params(params)
    ee1_m_1_params['lim_x'] = ee1_m_1_lim_x
    sols_EE1 = []
    sols_EE2 = []
    sols_EE3 = []

    # print(params, ee1_m_1_params)
    pb = progress_bar(range(M))
    for epoch in pb:
        sols_EE1.append(EE_1_plus_1(**ee1_m_1_params))
        sols_EE2.append(mu_lambda(**params))
        sols_EE3.append(mu_mas_lambda(**params))
    
    sols_EE1 = sorted(sols_EE1, key= lambda x : x[-1], reverse=True)
    sols_EE2 = sorted(sols_EE2, key= lambda x : x[-1], reverse=True)
    sols_EE3 = sorted(sols_EE3, key= lambda x : x[-1], reverse=True)

    mean_1 = mean(sols_EE1)
    mean_2 = mean(sols_EE2)
    mean_3 = mean(sols_EE3)

    besto_and_worse_sols  = [['EE 1 + 1', sols_EE1[0][0], sols_EE1[-1][0], round(mean(sols_EE1)), sols_EE1[len(sols_EE1)//2][0], np.round(std(sols_EE1, mean_1), decimals = 4)],
                             ['EE mu, lambda', sols_EE2[0][0], sols_EE2[-1][0], np.round(mean(sols_EE2), decimals= 4), sols_EE2[len(sols_EE2)//2][0], np.round(std(sols_EE2, mean_2), decimals = 4)],
                             ['EE mu + lambda', sols_EE3[0][0], sols_EE3[-1][0], round(mean(sols_EE3)), sols_EE3[len(sols_EE3)//2][0], np.round(std(sols_EE3, mean_3), decimals = 4)],
                            ]

    return_tuple = [format_data(besto_and_worse_sols)]
    if return_sols:
        return_tuple.append(sols_EE1)
        return_tuple.append(sols_EE2)
        return_tuple.append(sols_EE3)
    
    return tuple(return_tuple)


Analisis con $n = 2$ y demas valores default

In [25]:
print(statdistical(20, [[MIN_DOMAIN,MAX_DOMAIN], [0,10]]))

Estrategia Evolutiva    Mejor Solucion                                     Peor Solucion                                         Media  Mediana                                              Desviacion Estandar
----------------------  -------------------------------------------------  -------------------------------------------------  --------  -------------------------------------------------  ---------------------
EE 1 + 1                [-1.47406378  1.42175158]                          [28.49438125  0.        ]                          -18       [11.78654648  3.69441043]                                         3.0064
EE mu, lambda           [ 0.00071056 -0.0012165   0.01      ]              [ 2.26657053e-04 -7.52959046e-05  1.00000000e-02]    0.0017  [ 0.00039891 -0.00040984  0.01      ]                             0.0008
EE mu + lambda          [ 9.78960965e-05 -2.07871862e-04  1.00000000e-02]  [-3.26732111e-06  1.15651434e-05  1.00000000e-02]    0       [ 3.50809693e-05 -1.47650131

Analisis con $n = 5$

In [26]:
print(statdistical(M = 20,  ee1_m_1_lim_x = [[MIN_DOMAIN,MAX_DOMAIN]] * 5, n = 5))

Estrategia Evolutiva    Mejor Solucion                                                             Peor Solucion                                                                 Media  Mediana                                                                      Desviacion Estandar
----------------------  -------------------------------------------------------------------------  -------------------------------------------------------------------------  --------  -------------------------------------------------------------------------  ---------------------
EE 1 + 1                [10.38684492 -7.20456615  8.33945548 12.09203025  3.34276229]              [-21.55238655 -28.73130289  14.5879361  -24.605992     5.5771974 ]         -21       [ -8.45301307  15.44908906  -3.36897146   6.35734743 -26.20952888]                        0.7037
EE mu, lambda           [ 5.93291212e-03 -1.58144666e-05 -5.45882555e-03  2.84208016e-03           [-0.00010189  0.0019106  -0.00112411 -0.00034555 -0.000120

Analisis con $n = 7$

In [27]:
print(statdistical(M = 20, ee1_m_1_lim_x=[[MIN_DOMAIN, MAX_DOMAIN]] * 7,  n = 7))

Estrategia Evolutiva    Mejor Solucion                                                            Peor Solucion                                                                Media  Mediana                                                                     Desviacion Estandar
----------------------  ------------------------------------------------------------------------  ------------------------------------------------------------------------  --------  ------------------------------------------------------------------------  ---------------------
EE 1 + 1                [-17.86713021 -16.98406815 -26.99950226  -6.10129346 -29.27077386         [ -0.44598746   5.61759554 -28.7110843   22.6048886   29.34557573         -21       [  5.59974494 -21.14336066  17.76964424  22.62107275 -15.3190632                         0.2321
                         -25.17993066 -22.27099027]                                                 10.01969193  19.41985065]                                         

Analisis $n = 10$

In [28]:
print(statdistical(M = 20, ee1_m_1_lim_x=[[MIN_DOMAIN,MAX_DOMAIN]] * 10, n = 10))

Estrategia Evolutiva    Mejor Solucion                                                            Peor Solucion                                                          Media  Mediana                                                                     Desviacion Estandar
----------------------  ------------------------------------------------------------------------  ------------------------------------------------------------------  --------  ------------------------------------------------------------------------  ---------------------
EE 1 + 1                [ -2.12449469 -24.47399217   2.16406733  12.81930746   8.40973418         [-13.93663952  21.43816865  29.61403911  11.49315178  15.82400515   -21       [-29.12970449 -28.27571169  -0.12677957  10.70780987 -10.16635705                        0.4967
                         -11.28038218  15.84081489  14.83081442  -5.91793083   6.68854609]         -25.48720726 -29.13007453   8.65325087  -9.69442211 -17.57964986]              12.168

Analisis $n = 20$

In [29]:
print(statdistical(M = 20, ee1_m_1_lim_x=[[MIN_DOMAIN,MAX_DOMAIN]] * 20, n = 20))

Estrategia Evolutiva    Mejor Solucion                                                      Peor Solucion                                                                Media  Mediana                                                                     Desviacion Estandar
----------------------  ------------------------------------------------------------------  ------------------------------------------------------------------------  --------  ------------------------------------------------------------------------  ---------------------
EE 1 + 1                [ -6.49316983 -20.3991588  -22.05454188   5.79614485  -3.76606947   [-26.82403011 -22.59934181  29.53041756 -14.2934109  -24.02531175         -21       [ -1.89174765 -19.81151801  11.36588012   5.32952651   4.40066933                        0.2546
                           4.95575518 -21.63091251 -15.66661625   7.07355409  18.68687475    -21.6918728  -18.00150806 -20.71178429  15.59379053 -23.15950847                     -7.837